In [5]:
import plotly.express as px
import pandas as pd

ipca_url = "https://www.ipca.com/"
minimum_wage_url = "https://www.contabeis.com.br/tabelas/salario-minimo/"

ipca_df = pd.read_csv(ipca_url, index_col="Data")
ipca_df

ParserError: Error tokenizing data. C error: Expected 1 fields in line 6, saw 5


In [7]:
import plotly.express as px
import pandas as pd
from datetime import datetime

class BrasilPowerPurchase:

    def __init__(self):
        self.data = self.load_data()
        self.presidents = self.get_presidents()

    def load_data(self):
        # Obter dados do IPCA e Salário Mínimo
        ipca_df = self._get_ipca_data()
        minimum_wage_df = self._get_minimum_wage_data()

        # Combinar DataFrames
        df = pd.merge(ipca_df, minimum_wage_df, on=["Data"])

        # Calcular índice de poder de compra
        df["Poder de Compra"] = df["Salário Mínimo"] / df["IPCA"]

        return df

    def _get_ipca_data(self):
        # Acessar site do Banco Central e extrair dados do IPCA
        url = "https://www.bcb.gov.br/estatisticas/igpm"
        tables = pd.read_html(url)
        ipca_df = tables[1].iloc[1:, :]

        # Ajustar colunas e tipos de dados
        ipca_df.columns = ["Data", "IPCA"]
        ipca_df["Data"] = pd.to_datetime(ipca_df["Data"])
        ipca_df["IPCA"] = ipca_df["IPCA"].str.replace(",", ".").astype(float)

        return ipca_df

    def _get_minimum_wage_data(self):
        # Acessar site do Ministério da Economia e extrair dados do Salário Mínimo
        url = "https://www.gov.br/trabalho-e-previdencia/pt-br/composicao/orgaos-especiais/secretaria-de-trabalho/salario-minimo"
        tables = pd.read_html(url)
        minimum_wage_df = tables[0].iloc[2:, :]

        # Ajustar colunas e tipos de dados
        minimum_wage_df.columns = ["Ano", "Salário Mínimo"]
        minimum_wage_df["Ano"] = minimum_wage_df["Ano"].astype(int)
        minimum_wage_df["Salário Mínimo"] = minimum_wage_df["Salário Mínimo"].str.replace(",", ".").astype(float)

        # Converter Ano para Data
        minimum_wage_df["Data"] = minimum_wage_df["Ano"].apply(lambda year: datetime(year, 1, 1))

        return minimum_wage_df

    def get_presidents(self):
        # Criar dicionário com presidentes e seus mandatos
        presidents = {
            "1995-01-01": "Fernando Henrique Cardoso",
            "2003-01-01": "Luiz Inácio Lula da Silva",
            "2011-01-01": "Dilma Rousseff",
            "2016-08-31": "Michel Temer",
            "2019-01-01": "Jair Bolsonaro",
            "2023-01-01": "Luiz Inácio Lula da Silva"
        }

        return presidents

    def plot_power_purchase(self):
        # Criar figura
        fig = px.line(self.data, x="Data", y="Poder de Compra", title="Evolução do Poder de Compra no Brasil")

        # Adicionar linhas verticais interativas para transições presidenciais
        for year in self.presidents.keys():
            start_year = pd.to_datetime(year)
            fig.add_vline(x=start_year, line_width=1, line_color="gray", line_dash="dash")

        # Adicionar anotações com nome dos presidentes
        for year, president in self.presidents.items():
            start_year = pd.to_datetime(year)
            fig.add_annotation(
                x=start_year, y=self.data.loc[self.data["Data"] == start_year, "Poder de Compra"].iloc[0],
                text=president,
                font_size=12,
                font_color="black",
                textangle=-45
            )

        # Personalizar título, legendas e layout
        fig


In [9]:
# Executar script
brasil_power_purchase = BrasilPowerPurchase()
ipca_df = brasil_power_purchase._get_ipca_data()
ipca_df

ValueError: No tables found

In [ ]:
df

In [ ]:

brasil_power_purchase.plot_power_purchase()